In [1]:
# from IPython.display import GeoJSON

# GeoJSON({
#     "type": "Feature",
#     "geometry": {
#         "type": "Point",
#         "coordinates": [-118.4563712, 34.0163116]
#     }
# })

In [2]:
import sys
from data import api_key 
key=api_key.google_api_key

In [3]:
# load data
from glob import glob 
raw_data = glob("data/*.txt")
raw_data[0]

'data/KRI-DAC_Jeju_data6.txt'

In [4]:
# conncat data
from glob import glob 
concat_list=[]
for i in range(0,len(raw_data)):
    df=pd.read_csv(raw_data[i])
    df["month"]=raw_data[i].split(".")[0][-1:]
    concat_list.append(df)

In [6]:
df=pd.concat(concat_list)

In [8]:
#dropped useless columns 
df.columns=df.columns.str.lower()
df.drop(['sido','x',"objectid",'y'],axis = 1, inplace=True)

In [9]:
#rename koren named columns to eng
df

,field1,ym,sigungu,franclass,type,time,totalspent,disspent,numofspent,numofdisspent,point_x,point_y,month
0,1,202006,제주시,영세,일반한식,00시,502000,0,10,0,877005.9834,1.479766e+06,6
1,2,202006,제주시,영세,단란주점,00시,1520000,0,8,0,877005.7447,1.479816e+06,6
2,3,202006,제주시,중소1,편의점,00시,482310,0,35,0,877056.6756,1.479616e+06,6
3,4,202006,제주시,영세,편의점,00시,38050,5450,3,1,877055.9593,1.479766e+06,6
4,5,202006,제주시,영세,일반한식,00시,32000,32000,1,1,877055.4817,1.479866e+06,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
280080,280081,202008,제주시,영세,편의점,x시,13700,0,2,0,950266.5976,1.502458e+06,8
280081,280082,202008,제주시,영세,일반한식,x시,56000,0,1,0,950317.5379,1.502258e+06,8
280082,280083,202008,제주시,영세,일반한식,x시,28000,0,1,0,950469.1585,1.501909e+06,8
280083,280084,202008,제주시,중소1,중국음식,x시,46000,0,3,0,950574.6388,1.500760e+06,8


from pyproj import Proj, transform 
proj_itrf = Proj(init='epsg:5178')
proj_wgs84 = Proj(init='epsg:4326')
df["lon"],df["lat"] = transform(proj_itrf,proj_wgs84,df['point_x'],df["point_y"])

df['lon_lat']=df['lon'].astype(str)+'_'+df['lat'].astype(str)

df_lon_lat=df['lon_lat'].drop_duplicates()
df_lon_lat

import googlemaps
from datetime import datetime

gmaps = googlemaps.Client(key) 
reverse_geocode_result = gmaps.reverse_geocode((33.302315,126.176746))
reverse_geocode_result
# geocode_result  

import json
m=folium.Map(location=[48,-102],zoom_start=3)
folium.Choropleth(
    geo_data='./data/03_us-states.json',
    data=state_data, columns=["State",'Unemployment'],
    nan_fill_color="purple", nan_fill_opacity=0.4,
    key_on="    ", fill_color="YlGn",
    legend_name="Unemployment Rate (%)"
).add_to(m)
m

In [19]:
#import rename module  
import type_names
input_data=type_names.rename()
for key, value in input_data:
    df["type"].replace(dict.fromkeys(key, value), inplace=True)

In [21]:
df

,field1,ym,sigungu,franclass,type,time,totalspent,disspent,numofspent,numofdisspent,point_x,point_y,month,lon,lat,lon_lat
0,1,202006,제주시,영세,외식,00시,502000,0,10,0,877005.9834,1.479766e+06,6,126.176746,33.305090,126.17674597473471_33.30508967004505
1,2,202006,제주시,영세,유흥/주점,00시,1520000,0,8,0,877005.7447,1.479816e+06,6,126.176737,33.305540,126.17673661244419_33.305540299877514
2,3,202006,제주시,중소1,식료품,00시,482310,0,35,0,877056.6756,1.479616e+06,6,126.177311,33.303746,126.17731071479301_33.30374564295666
3,4,202006,제주시,영세,식료품,00시,38050,5450,3,1,877055.9593,1.479766e+06,6,126.177283,33.305098,126.17728263513888_33.30509753116355
4,5,202006,제주시,영세,외식,00시,32000,32000,1,1,877055.4817,1.479866e+06,6,126.177264,33.305999,126.17726391376034_33.30599879008531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280080,280081,202008,제주시,영세,식료품,x시,13700,0,2,0,950266.5976,1.502458e+06,8,126.962369,33.515632,126.96236864779921_33.51563161134591
280081,280082,202008,제주시,영세,외식,x시,56000,0,1,0,950317.5379,1.502258e+06,8,126.962928,33.513833,126.96292823685066_33.5138330296567
280082,280083,202008,제주시,영세,외식,x시,28000,0,1,0,950469.1585,1.501909e+06,8,126.964580,33.510691,126.96458009867118_33.51069110892843
280083,280084,202008,제주시,중소1,외식,x시,46000,0,3,0,950574.6388,1.500760e+06,8,126.965779,33.500332,126.96577927205708_33.50033236682921
